In [ ]:
from google.colab import drive
drive.mount('/content/Mydrive')
%cd "/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon"

In [ ]:
grid = gpd.GeoDataFrame.from_file('Seoul_Grid/grid.shp') # (절대)경로 개별적 수정

In [ ]:
!pip install geopandas

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [ ]:
# 사고데이터
df = gpd.read_file('Preprocessing/accidentdata.gpkg')
df.datetime = pd.to_datetime(df.datetime)
df = df.to_crs('epsg:5179')
df.head(1)

# SDoT 데이터 병합
- 유동인구 데이터 : 사고지점으로부터 가장 가까운 K개의 센서 데이터의 평균을 이용
- 지하철, 버스정류장 정보 : 사고지점으로부터 1000m 이내의 정류장/역의 개수

In [ ]:
K=3

In [ ]:
# SDoT 좌표
sdot_pop_coord = pd.read_csv('Preprocessing/전처리완료/SDoT_pop_Coordinate.csv', usecols=['modl_serial','la','lo'])
sdot_pop_coord = gpd.GeoDataFrame(sdot_pop_coord,geometry = gpd.points_from_xy(x = sdot_pop_coord.lo, y= sdot_pop_coord.la, crs='WGS84')).to_crs('epsg:5179')

In [ ]:
sdot_pop = pd.read_csv('/content/Mydrive/MyDrive/IOT_hackathon/Github/IOT_hackathon/data/S-DoT 유동인구 2021/SDOT_population.csv')
sdot_pop.등록일자 = pd.to_datetime(sdot_pop.등록일자)

In [ ]:
# 가장 가까운 K개의 센서 평균(K-means)으로 데이터셋 구성하기
df['SDoT_pop'] = 0.0
for i in tqdm(range(len(df))):
    arr = sdot_pop_coord.geometry.distance(df.geometry[i])
    idxs = (-arr).argsort()
    cnt = 0
    cnt_k = 0
    for j in idxs:
        idx = sdot_pop_coord.iloc[j,0]
        num_i = sdot_pop.loc[(sdot_pop['사이트명'] == int(idx))&(sdot_pop['등록일자'] == df.datetime[i]),'방문자수']
        if len(num_i) > 0 : 
            cnt_k += 1
            cnt += num_i.values[0]
            if cnt_k == 3:
                break
        if cnt_k == 0:
            cnt = 0
        else:
            cnt = cnt/cnt_k
    df.SDoT_pop[i] = cnt

In [ ]:
# 지하철, 버스
bus_loc = pd.read_csv('Preprocessing/전처리완료/busstop_location.csv')
subway_loc = pd.read_csv('Preprocessing/전처리완료/subway_location.csv')

In [ ]:
bus_loc

In [ ]:
bus_loc = gpd.GeoDataFrame(bus_loc,geometry = gpd.points_from_xy(x = bus_loc.CRDNT_X, y= bus_loc.CRDNT_Y, crs='WGS84')).to_crs('epsg:5179')
subway_loc = gpd.GeoDataFrame(subway_loc,geometry = gpd.points_from_xy(x = subway_loc.CONV_X, y= subway_loc.CONV_Y, crs='WGS84')).to_crs('epsg:5179')

In [ ]:
# 가장 가까운 K개의 센서 평균(K-means)으로 데이터셋 구성하기
df['subway'] = 0
df['bus'] = 0
for i in tqdm(range(len(df))):
    arr_sub = subway_loc.geometry.distance(df.geometry[i])
    arr_bus = bus_loc.geometry.distance(df.geometry[i])
    idx_sub = arr_sub[arr_sub<1000].index.values
    idx_bus = arr_bus[arr_bus<1000].index.values
    df.subway[i] = len(idx_sub)
    df.bus[i] = len(idx_bus)

### S-DoT 환경데이터 처리
- 각 사고지점에 가장 가까운 센서의 데이터를 포함시킴

In [ ]:
# SDoT 환경센서 좌표
sdot_env_coord = pd.read_csv('Preprocessing/전처리완료/SDoT_env_Coordinate.csv', usecols=['modl_serial','la','lo'])
sdot_env_coord = gpd.GeoDataFrame(sdot_env_coord,geometry = gpd.points_from_xy(x = sdot_env_coord.lo, y= sdot_env_coord.la, crs='WGS84')).to_crs('epsg:5179')

In [ ]:
# S-DoT 환경 데이터셋
sdot_env = pd.read_csv('Preprocessing/전처리완료/SDOT_Env_pros_1.csv')
sdot_env.등록일자 = pd.to_datetime(sdot_env.등록일자)

In [ ]:
df['temp'] = 0.0
df['humidity'] = 0.0
df['dust'] = 0.0
for i in tqdm(range(len(df))):
    arr = sdot_env_coord.geometry.distance(df.geometry[i])
    idxs = (-arr).argsort()
    j = 0
    while True:
        idx = sdot_env_coord.iloc[idxs[j],0]
        num_t = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '기온(℃)']
        num_h = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '상대습도( %)']
        num_d = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '미세먼지 보정(㎍/㎥)']    
        if len(num_t) == 0:
            j += 1
        else:
            break
        
        if j > 10:
            break
    
    if len(num_t) > 0:
        df.temp[i] = num_t.values[0]
        df.humidity[i] = num_h.values[0]
        df.dust[i] = num_d.values[0]
    else:
        df.temp[i] = np.nan
        df.humidity[i] = np.nan
        df.dust[i] = np.nan
    
   
    # for j in idxs:
    #     idx = sdot_env_coord.iloc[j,0]
    #     num_t = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '기온(℃)']
    #     num_h = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '상대습도( %)']
    #     num_d = sdot_env.loc[(sdot_env['시리얼'] == idx)&(sdot_env['등록일자'] == df.datetime[i]), '미세먼지 보정(㎍/㎥)']

    #     if len(num_t) > 0:
    #         cnt_k += 1
    #         temp += num_t.values[0]
    #         # humidity += num_h
    #         # dust += num_d
            
    #         if cnt_k == K:
    #             break
    #     if cnt_k == 0:
    #         temp = 0
    #         # humidity = 0
    #         # dust = 0
    #     else:
    #         temp = temp/cnt_k
    #         # humidity = humidity/cnt_k
    #         # dust = dust/cnt_k

    # df['temp'][i] = temp
    # # df.humidity[i] = humidity
    # # df.dust[i] = dust

### 미리 작성된 격자-시계열형 데이터셋에서 기타 변수들 추출
- `final.pkl` 파일로 저장된 데이터셋에서 각 사고 일시 및 좌표가 속한 격자에 맞는 변수값 추출

In [ ]:
df_final = pd.read_pickle('final.pkl')

In [ ]:
df[df_final.columns.drop(['geometry','dt','accident'])] = 0

In [ ]:
df.columns

In [ ]:
for i in tqdm(range(len(df))):
    dt_i = df.datetime[i]
    idx = np.where(grid.geometry.contains(df.geometry[i]))[0][0]
    ls = df_final.loc[df_final.index == idx,:].iloc[0,3:]
    df.iloc[i,25:] = ls

In [ ]:
df = df[df.columns.drop(['subway'])]

In [ ]:
df.to_csv('Dataset.csv')